In [1]:
"""
NHL DATA AUDIT SCRIPT
Purpose: Understand what data we have from each source and identify gaps
"""

import pandas as pd
import requests
from datetime import datetime
import json

In [2]:
# ============================================================
# CONFIGURATION
# ============================================================

MONEYPUCK_CSV_PATH = "../data/raw/moneypuck_skaters_current.csv"  # UPDATE THIS
ODDS_API_KEY = "b81efa2d2e27c141b788a7fb02065fe0"  # UPDATE THIS

In [4]:
# ============================================================
# 1. AUDIT MONEYPUCK CSV
# ============================================================

def audit_moneypuck():
    """Check what MoneyPuck gives us and how fresh it is"""
    print("=" * 60)
    print("MONEYPUCK CSV AUDIT")
    print("=" * 60)
    
    df = pd.read_csv(MONEYPUCK_CSV_PATH)
    
    print(f"\n📊 SHAPE: {df.shape[0]} rows × {df.shape[1]} columns")
    print(f"\n📋 COLUMNS:")
    for i, col in enumerate(df.columns, 1):
        print(f"  {i}. {col}")
    
    # Check for date columns to assess freshness
    date_cols = [col for col in df.columns if 'date' in col.lower() or 'game' in col.lower()]
    if date_cols:
        print(f"\n📅 DATE COLUMNS: {date_cols}")
        for col in date_cols:
            try:
                print(f"  Latest in {col}: {df[col].max()}")
            except:
                print(f"  {col}: Cannot parse dates")
    
    # Check for key columns we care about
    key_columns = {
        'player': ['name', 'player', 'player_name'],
        'xG': ['xg', 'ixg', 'expected_goals', 'xgoals'],
        'PP': ['pp', 'powerplay', 'pp_toi', 'pp1'],
        'TOI': ['toi', 'ice_time', 'time_on_ice'],
        'shots': ['shots', 'isf', 'shots_on_goal']
    }
    
    print(f"\n🔍 KEY FEATURES PRESENT:")
    for category, possible_names in key_columns.items():
        found = [col for col in df.columns if any(name in col.lower() for name in possible_names)]
        status = "✅" if found else "❌"
        print(f"  {status} {category}: {found if found else 'NOT FOUND'}")
    
    # Sample rows
    print(f"\n📝 SAMPLE DATA (first 3 rows):")
    print(df.head(3).to_string())
    
    return df

In [5]:
# ============================================================
# 2. AUDIT NHL OFFICIAL API
# ============================================================

def audit_nhl_api():
    """Check what NHL official API gives us for TODAY's games"""
    print("\n" + "=" * 60)
    print("NHL OFFICIAL API AUDIT")
    print("=" * 60)
    
    # Get today's schedule
    print("\n📅 FETCHING TODAY'S SCHEDULE...")
    url = "https://api-web.nhle.com/v1/schedule/now"
    response = requests.get(url)
    
    if response.status_code != 200:
        print(f"❌ Failed: {response.status_code}")
        return None
    
    data = response.json()
    
    # Parse games
    games_today = []
    if 'gameWeek' in data:
        for day in data['gameWeek']:
            if 'games' in day:
                games_today.extend(day['games'])
    
    print(f"\n🏒 GAMES TODAY: {len(games_today)}")
    
    if not games_today:
        print("⚠️ No games today - using a recent game for structure audit")
        # Fallback to get a recent game
        url = "https://api-web.nhle.com/v1/schedule/2024-12-23"  # Use recent date
        response = requests.get(url)
        data = response.json()
        if 'gameWeek' in data:
            for day in data['gameWeek']:
                if 'games' in day:
                    games_today.extend(day['games'])
    
    if not games_today:
        print("❌ Could not fetch any games")
        return None
    
    # Show first game structure
    game = games_today[0]
    print(f"\n📋 GAME STRUCTURE (first game):")
    print(f"  Game ID: {game.get('id')}")
    print(f"  Matchup: {game.get('awayTeam', {}).get('abbrev')} @ {game.get('homeTeam', {}).get('abbrev')}")
    print(f"  Game State: {game.get('gameState')}")
    
    # Try to get detailed game data
    game_id = game.get('id')
    if game_id:
        print(f"\n🔍 FETCHING DETAILED GAME DATA...")
        detail_url = f"https://api-web.nhle.com/v1/gamecenter/{game_id}/play-by-play"
        detail_response = requests.get(detail_url)
        
        if detail_response.status_code == 200:
            detail_data = detail_response.json()
            
            # Check for roster/lineup info
            print(f"\n📋 AVAILABLE KEYS IN GAME DATA:")
            for key in detail_data.keys():
                print(f"  - {key}")
            
            # Check for rosters
            if 'rosterSpots' in detail_data:
                print(f"\n👥 ROSTER EXAMPLE (first 5 players):")
                roster = detail_data['rosterSpots'][:5]
                for player in roster:
                    print(f"  {player.get('firstName', {}).get('default')} {player.get('lastName', {}).get('default')} - Position: {player.get('positionCode')}")
    
    # Get player stats structure
    print(f"\n🔍 TESTING PLAYER STATS ENDPOINT...")
    # Using Connor McDavid as example (ID: 8478402)
    player_url = "https://api-web.nhle.com/v1/player/8478402/landing"
    player_response = requests.get(player_url)
    
    if player_response.status_code == 200:
        player_data = player_response.json()
        print(f"\n📋 PLAYER DATA STRUCTURE:")
        for key in player_data.keys():
            print(f"  - {key}")
        
        if 'featuredStats' in player_data:
            print(f"\n📊 AVAILABLE STATS:")
            stats = player_data['featuredStats']
            if 'regularSeason' in stats and 'subSeason' in stats['regularSeason']:
                season_stats = stats['regularSeason']['subSeason']
                for stat_key in season_stats.keys():
                    print(f"  - {stat_key}")
    
    return games_today

In [6]:
# ============================================================
# 3. AUDIT ODDSAPI STRUCTURE
# ============================================================

def audit_oddsapi(api_key):
    """Check what OddsAPI gives us"""
    print("\n" + "=" * 60)
    print("ODDSAPI AUDIT")
    print("=" * 60)
    
    print("\n📅 FETCHING NHL ODDS...")
    url = "https://api.the-odds-api.com/v4/sports/icehockey_nhl/odds"
    params = {
        'apiKey': api_key,
        'regions': 'us',
        'markets': 'player_goal_scorer',
        'oddsFormat': 'decimal'
    }
    
    response = requests.get(url, params=params)
    
    if response.status_code != 200:
        print(f"❌ Failed: {response.status_code}")
        return None
    
    data = response.json()
    
    print(f"\n🏒 EVENTS WITH ODDS: {len(data)}")
    print(f"📊 Requests Remaining: {response.headers.get('x-requests-remaining', 'Unknown')}")
    
    if data:
        event = data[0]
        print(f"\n📋 EVENT STRUCTURE:")
        print(f"  ID: {event.get('id')}")
        print(f"  Matchup: {event.get('away_team')} @ {event.get('home_team')}")
        print(f"  Start: {event.get('commence_time')}")
        
        if 'bookmakers' in event and event['bookmakers']:
            bookmaker = event['bookmakers'][0]
            print(f"\n📋 BOOKMAKER STRUCTURE:")
            print(f"  Name: {bookmaker.get('key')}")
            print(f"  Last Update: {bookmaker.get('last_update')}")
            
            if 'markets' in bookmaker and bookmaker['markets']:
                market = bookmaker['markets'][0]
                print(f"\n📋 MARKET STRUCTURE:")
                print(f"  Key: {market.get('key')}")
                
                if 'outcomes' in market and market['outcomes']:
                    print(f"  Player Props Available: {len(market['outcomes'])}")
                    print(f"\n📝 SAMPLE PROPS (first 5):")
                    for outcome in market['outcomes'][:5]:
                        print(f"    {outcome.get('description')} @ {outcome.get('price')}")
    
    return data

In [7]:
# ============================================================
# 4. CREATE COMPARISON TABLE
# ============================================================

def create_comparison_table():
    """Summary table of what each source provides"""
    print("\n" + "=" * 60)
    print("DATA SOURCE COMPARISON")
    print("=" * 60)
    
    comparison = pd.DataFrame({
        'Data Source': ['MoneyPuck CSV', 'NHL Official API', 'OddsAPI'],
        'Player Quality (xG, etc)': ['✅ Historical', '✅ Season totals', '❌'],
        'Current Lineup': ['❌ Lagged', '✅ Real-time', '⚠️ Lists all, not filtered'],
        'PP Unit Assignment': ['⚠️ Historical only', '⚠️ Inferred from TOI', '❌'],
        'Opponent/Matchup': ['❌', '✅ Game details', '✅ Teams only'],
        'Betting Odds': ['❌', '❌', '✅'],
        'Update Frequency': ['Daily/Weekly', 'Real-time', 'Real-time'],
        'Best Used For': ['Player skill priors', 'Lineup gate', 'Market prices']
    })
    
    print("\n")
    print(comparison.to_string(index=False))
    
    print("\n" + "=" * 60)
    print("RECOMMENDED DATA PIPELINE")
    print("=" * 60)
    print("""
1. NHL API (GATE) → Who is playing tonight?
   └─ Get today's games + rosters
   └─ HARD FILTER: Only players in confirmed lineup

2. MoneyPuck (PRIORS) → Player quality metrics
   └─ Get player xG, shot rates, historical performance
   └─ Use as base probability

3. OddsAPI (MARKET) → Get odds for players who passed gate
   └─ Calculate implied probability
   └─ Compare to model probability

4. FINAL MODEL FEATURES (in order):
   ✅ PP1 indicator (manual CSV or infer from TOI)
   ✅ TOI rolling average
   ✅ ixG rolling average
   ✅ Home/Away
   ✅ Opponent quality (goals against)
    """)

In [8]:
# ============================================================
# 5. RUN ALL AUDITS
# ============================================================

if __name__ == "__main__":
    print("\n" + "🔍" * 30)
    print("NHL BETTING MODEL - DATA AUDIT")
    print("🔍" * 30)
    print(f"\nRun Time: {datetime.now()}")
    
    # Run audits
    try:
        moneypuck_df = audit_moneypuck()
    except Exception as e:
        print(f"⚠️ MoneyPuck audit failed: {e}")
        moneypuck_df = None
    
    try:
        nhl_games = audit_nhl_api()
    except Exception as e:
        print(f"⚠️ NHL API audit failed: {e}")
        nhl_games = None
    
    try:
        odds_data = audit_oddsapi(ODDS_API_KEY)
    except Exception as e:
        print(f"⚠️ OddsAPI audit failed: {e}")
        odds_data = None
    
    # Create comparison
    create_comparison_table()
    
    print("\n" + "=" * 60)
    print("AUDIT COMPLETE")
    print("=" * 60)
    print("\n🎯 NEXT STEPS:")
    print("1. Review column names from each source")
    print("2. Identify which CSV has what features")
    print("3. Build lineup gate using NHL API")
    print("4. Add PP1/PP2 manually if not in data")
    print("5. Re-run model with filtered players only")


🔍🔍🔍🔍🔍🔍🔍🔍🔍🔍🔍🔍🔍🔍🔍🔍🔍🔍🔍🔍🔍🔍🔍🔍🔍🔍🔍🔍🔍🔍
NHL BETTING MODEL - DATA AUDIT
🔍🔍🔍🔍🔍🔍🔍🔍🔍🔍🔍🔍🔍🔍🔍🔍🔍🔍🔍🔍🔍🔍🔍🔍🔍🔍🔍🔍🔍🔍

Run Time: 2025-12-23 19:49:03.534921
MONEYPUCK CSV AUDIT

📊 SHAPE: 4135 rows × 154 columns

📋 COLUMNS:
  1. playerId
  2. season
  3. name
  4. team
  5. position
  6. situation
  7. games_played
  8. icetime
  9. shifts
  10. gameScore
  11. onIce_xGoalsPercentage
  12. offIce_xGoalsPercentage
  13. onIce_corsiPercentage
  14. offIce_corsiPercentage
  15. onIce_fenwickPercentage
  16. offIce_fenwickPercentage
  17. iceTimeRank
  18. I_F_xOnGoal
  19. I_F_xGoals
  20. I_F_xRebounds
  21. I_F_xFreeze
  22. I_F_xPlayStopped
  23. I_F_xPlayContinuedInZone
  24. I_F_xPlayContinuedOutsideZone
  25. I_F_flurryAdjustedxGoals
  26. I_F_scoreVenueAdjustedxGoals
  27. I_F_flurryScoreVenueAdjustedxGoals
  28. I_F_primaryAssists
  29. I_F_secondaryAssists
  30. I_F_shotsOnGoal
  31. I_F_missedShots
  32. I_F_blockedShotAttempts
  33. I_F_shotAttempts
  34. I_F_points
  35. I_F_goals
  36. I_F_rebounds
  

In [ ]:
"""
GET CURRENT SEASON (2025-26) STATS FROM NHL API
Add this cell to your notebook
"""

import pandas as pd
import requests
from datetime import datetime
import time

# ============================================================
# 1. GET CURRENT SEASON PLAYER STATS
# ============================================================

def get_current_season_stats():
    """
    Fetch 2025-26 season stats from NHL API
    This replaces stale MoneyPuck data
    """
    print("=" * 60)
    print("FETCHING 2025-26 SEASON STATS FROM NHL API")
    print("=" * 60)
    
    # Get all teams first
    teams_url = "https://api.nhle.com/stats/rest/en/team"
    teams_response = requests.get(teams_url)
    
    if teams_response.status_code != 200:
        print("❌ Could not fetch teams")
        return None
    
    teams = teams_response.json().get('data', [])
    team_abbrevs = [team['triCode'] for team in teams if 'triCode' in team]
    
    print(f"✅ Found {len(team_abbrevs)} teams")
    
    all_players = []
    
    # Get roster for each team
    for team_abbrev in team_abbrevs:
        print(f"📥 Fetching {team_abbrev}...", end=" ")
        
        roster_url = f"https://api-web.nhle.com/v1/roster/{team_abbrev}/current"
        roster_response = requests.get(roster_url)
        
        if roster_response.status_code != 200:
            print("❌")
            continue
        
        roster_data = roster_response.json()
        
        # Get stats for each player
        for position_group in ['forwards', 'defensemen']:
            if position_group not in roster_data:
                continue
                
            for player in roster_data[position_group]:
                player_id = player.get('id')
                
                # Fetch detailed player stats
                player_url = f"https://api-web.nhle.com/v1/player/{player_id}/landing"
                player_response = requests.get(player_url)
                
                if player_response.status_code != 200:
                    continue
                
                player_data = player_response.json()
                
                # Extract current season stats
                if 'featuredStats' in player_data:
                    stats = player_data['featuredStats']
                    if 'regularSeason' in stats and 'subSeason' in stats['regularSeason']:
                        season_stats = stats['regularSeason']['subSeason']
                        
                        player_info = {
                            'playerId': player_id,
                            'name': f"{player.get('firstName', {}).get('default', '')} {player.get('lastName', {}).get('default', '')}",
                            'team': team_abbrev,
                            'position': player.get('positionCode'),
                            'season': '2025-26',
                            'games_played': season_stats.get('gamesPlayed', 0),
                            'goals': season_stats.get('goals', 0),
                            'assists': season_stats.get('assists', 0),
                            'points': season_stats.get('points', 0),
                            'shots': season_stats.get('shots', 0),
                            'shooting_pct': season_stats.get('shootingPctg', 0),
                            'pp_goals': season_stats.get('powerPlayGoals', 0),
                            'pp_points': season_stats.get('powerPlayPoints', 0),
                            'sh_goals': season_stats.get('shorthandedGoals', 0),
                            'gwg': season_stats.get('gameWinningGoals', 0),
                            'pim': season_stats.get('pim', 0),
                            'plus_minus': season_stats.get('plusMinus', 0)
                        }
                        
                        all_players.append(player_info)
                
                time.sleep(0.05)  # Rate limiting
        
        print(f"✅ ({len([p for p in all_players if p['team'] == team_abbrev])} players)")
    
    df = pd.DataFrame(all_players)
    print(f"\n✅ TOTAL: {len(df)} players with 2025-26 stats")
    
    return df

# ============================================================
# 2. CALCULATE SIMPLE xG PROXY FROM NHL STATS
# ============================================================

def calculate_simple_xg(df):
    """
    Create simple xG estimates from NHL API data
    Since we don't have MoneyPuck's xG for current season
    """
    print("\n" + "=" * 60)
    print("CALCULATING SIMPLE xG PROXY")
    print("=" * 60)
    
    # Calculate goals per game
    df['goals_per_game'] = df['goals'] / df['games_played'].replace(0, 1)
    
    # Calculate shots per game
    df['shots_per_game'] = df['shots'] / df['games_played'].replace(0, 1)
    
    # Simple xG estimate: regression to mean between actual goals and league average
    league_avg_goals_per_game = df[df['games_played'] > 5]['goals_per_game'].mean()
    
    # Weight actual performance by games played (more games = more confidence)
    games_weight = df['games_played'] / (df['games_played'] + 10)  # 10 game prior
    df['simple_xg_per_game'] = (
        games_weight * df['goals_per_game'] + 
        (1 - games_weight) * league_avg_goals_per_game
    )
    
    # Identify PP1 players (top PP performers by team)
    df['pp_goals_per_game'] = df['pp_goals'] / df['games_played'].replace(0, 1)
    df['pp_rank_in_team'] = df.groupby('team')['pp_points'].rank(ascending=False)
    df['is_pp1'] = (df['pp_rank_in_team'] <= 5).astype(int)  # Top 5 = PP1
    
    # Calculate recent form (this is placeholder - would need game logs for true rolling average)
    df['form_weight'] = 1.0  # Would calculate from last 5-10 games
    
    print(f"✅ Calculated xG proxy for {len(df)} players")
    print(f"\n📊 Stats Summary:")
    print(f"  Avg goals/game: {df['goals_per_game'].mean():.3f}")
    print(f"  Avg shots/game: {df['shots_per_game'].mean():.3f}")
    print(f"  PP1 players: {df['is_pp1'].sum()}")
    
    return df

# ============================================================
# 3. ENHANCED: GET GAME LOGS FOR ROLLING AVERAGES
# ============================================================

def get_recent_form(player_id, last_n_games=5):
    """
    Get last N games for rolling averages
    This is more accurate than season totals
    """
    # NHL API game logs endpoint
    gamelog_url = f"https://api-web.nhle.com/v1/player/{player_id}/game-log/now"
    
    try:
        response = requests.get(gamelog_url)
        if response.status_code != 200:
            return None
        
        data = response.json()
        
        if 'gameLog' not in data or not data['gameLog']:
            return None
        
        games = data['gameLog'][:last_n_games]  # Last N games
        
        recent_stats = {
            'recent_goals': sum(g.get('goals', 0) for g in games),
            'recent_shots': sum(g.get('shots', 0) for g in games),
            'recent_games': len(games),
            'recent_toi_avg': sum(g.get('toi', '0:00').split(':')[0] for g in games) / len(games) if games else 0
        }
        
        return recent_stats
    except:
        return None

# ============================================================
# 4. INTEGRATE WITH YOUR EXISTING PIPELINE
# ============================================================

def create_current_season_model_data():
    """
    Complete pipeline: Get current season data + merge with today's games
    """
    print("\n" + "🚀" * 30)
    print("BUILDING CURRENT SEASON MODEL DATA")
    print("🚀" * 30)
    
    # Step 1: Get current season stats
    current_stats_df = get_current_season_stats()
    
    if current_stats_df is None:
        print("❌ Failed to get current season stats")
        return None
    
    # Step 2: Calculate xG proxies
    current_stats_df = calculate_simple_xg(current_stats_df)
    
    # Step 3: Get today's lineups (reuse from previous code)
    print("\n" + "=" * 60)
    print("GETTING TODAY'S LINEUPS")
    print("=" * 60)
    
    schedule_url = "https://api-web.nhle.com/v1/schedule/now"
    response = requests.get(schedule_url)
    
    if response.status_code != 200:
        print("❌ Could not get today's schedule")
        return current_stats_df
    
    data = response.json()
    games_today = []
    
    if 'gameWeek' in data:
        for day in data['gameWeek']:
            if 'games' in day:
                games_today.extend(day['games'])
    
    print(f"🏒 Games today: {len(games_today)}")
    
    # Extract teams playing today
    teams_today = set()
    for game in games_today:
        teams_today.add(game.get('awayTeam', {}).get('abbrev'))
        teams_today.add(game.get('homeTeam', {}).get('abbrev'))
    
    # Filter to only players on teams playing today
    todays_players = current_stats_df[current_stats_df['team'].isin(teams_today)].copy()
    
    print(f"\n✅ {len(todays_players)} players on teams playing today")
    
    # Save outputs
    current_stats_df.to_csv('nhl_2025_26_season_stats.csv', index=False)
    todays_players.to_csv('todays_players_current_season.csv', index=False)
    
    print(f"\n💾 Saved:")
    print(f"  - nhl_2025_26_season_stats.csv ({len(current_stats_df)} players)")
    print(f"  - todays_players_current_season.csv ({len(todays_players)} players)")
    
    return todays_players

# ============================================================
# RUN IT
# ============================================================

if __name__ == "__main__":
    # This will take 5-10 minutes to run (fetching all player data)
    todays_data = create_current_season_model_data()
    
    print("\n" + "=" * 60)
    print("✅ CURRENT SEASON DATA READY")
    print("=" * 60)
    print("\n🎯 NEXT STEPS:")
    print("1. Use todays_players_current_season.csv instead of MoneyPuck")
    print("2. Merge with OddsAPI data")
    print("3. Calculate probabilities with CURRENT season stats")
    print("4. Your EVs should be much more realistic now!")